In [ ]:
!pip install requests beautifulsoup4


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
!pip install scikit-learn rank-bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install Flask beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import re


import os
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer






In [ ]:
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from flask import Flask, render_template, request
from bs4 import BeautifulSoup
import requests
import os

app = Flask(__name__)






class WebCrawler:
    def __init__(self, seeds):
        self.seeds = seeds
        self.url_frontier = list(seeds)
        self.visited_urls = set()
        self.crawled_pages = []

    def crawl(self, max_pages):
        while self.url_frontier and len(self.crawled_pages) < max_pages:
            url = self.url_frontier.pop(0)
            if url not in self.visited_urls:
                content = self.fetch_page_content(url)
                if content is not None:
                    self.save_page(url, content)
                    new_links = self.extract_links(url, content)
                    self.add_new_links(new_links)
                    self.visited_urls.add(url)

    def fetch_page_content(self, url):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                return response.text
        except requests.exceptions.RequestException as e:
            print("Error fetching page:", e)
        return None

    def save_page(self, url, content):
        self.crawled_pages.append((url, content))
        filename = os.path.basename(url) + ".html"  # Extract filename from the URL
        with open(filename, "w", encoding="utf-8") as file:
            file.write(content)
        print("Page saved:", filename)

    def extract_links(self, url, content):
        links = []
        soup = BeautifulSoup(content, 'html.parser')
        for link in soup.find_all('a'):
            href = link.get('href')
            if href:
                absolute_url = urljoin(url, href)
                links.append(absolute_url)
        return links

    def add_new_links(self, links):
        for link in links:
            if link not in self.visited_urls and link not in self.url_frontier:
                self.url_frontier.append(link)


# Initialize web crawler
seeds = [
    'https://en.wikipedia.org/wiki/Cricket',
    'https://en.wikipedia.org/wiki/Cricket_(insect)',
    'https://en.wikipedia.org/wiki/Cricket_(disambiguation)',
    'https://en.wikipedia.org/wiki/New_York_City',
    'https://en.wikipedia.org/wiki/Box_score_(baseball)',
    'https://en.wikipedia.org/wiki/Cricket#cite_note-158',
    'https://en.wikipedia.org/wiki/New_York_Clipper',
    'https://en.wikipedia.org/wiki/Cricket#cite_note-NPR-159',
    'https://en.wikipedia.org/wiki/Cricket#cite_note-Myth-160',
    'https://en.wikipedia.org/wiki/Portal:Cricket',
]

max_pages = 100
crawler = WebCrawler(seeds)
crawler.crawl(max_pages)


# Text Extraction and Preprocessing
documents = []
folder_path = "/content/sample_data"
for url, content in crawler.crawled_pages:
    soup = BeautifulSoup(content, 'html.parser')
    title = soup.find('title')
    body = soup.find('body')
    if title and body:
        document = title.get_text() + " " + body.get_text()
        documents.append(document)

'''for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    if file_name.endswith('.html'):
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
            soup = BeautifulSoup(html_content, 'html.parser')
            title = soup.find('h1').get_text()
            body = soup.find('div', {'id': 'bodyContent'}).get_text()
            document = Document(title, body)
            documents.append(document)'''

# Search Engine
# Indexing
class Indexer:
    def __init__(self):
        self.vectorizer = None
        self.index = None

    def build_index(self, documents):
        self.vectorizer = CountVectorizer()
        self.index = self.vectorizer.fit_transform(documents)

indexer = Indexer()
if documents:
    indexer.build_index(documents)
else:
    print("No documents found for indexing.")


    def preprocess_text(self, text):
        tokens = word_tokenize(text)  # Tokenization
        tokens = [token.lower() for token in tokens]  # Convert tokens to lowercase
        tokens = [token for token in tokens if token.isalpha()]  # Remove non-alphabetic tokens
        tokens = [token for token in tokens if token not in self.stop_words]  # Remove stopwords
        preprocessed_text = ' '.join(tokens)
        return preprocessed_text

    def search(self, query, ranking_method, k=10):
        preprocessed_query = self.preprocess_text(query)
        query_vector = self.vectorizer.transform([preprocessed_query])
        if ranking_method == 'cosine_similarity':
            scores = cosine_similarity(query_vector, self.index)
            results = [(score, doc) for score, doc in zip(scores[0], self.documents)]
        elif ranking_method == 'bm25':
            corpus = [doc.split() for doc in self.documents]
            bm25 = BM25Okapi(corpus)
            scores = bm25.get_scores(preprocessed_query.split())
            results = [(score, doc) for score, doc in zip(scores, self.documents)]
        else:
            raise ValueError("Invalid ranking method. Choose 'cosine_similarity' or 'bm25'.")

        results.sort(reverse=True)
        top_k_results = results[:k]
        return top_k_results


# Initialize search engine and build index
indexer = Indexer()
indexer.build_index(documents)
crawler = WebCrawler(seeds)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/search', methods=['POST'])
def search():
    query = request.form['query']
    ranking_method = 'cosine_similarity'  # or 'bm25'
    results = indexer.search(query, ranking_method, k=10)

    # Retrieve document summaries
    summaries = []
    for _, doc in results:
        # Extract a brief summary of the document (e.g., first few sentences)
        # Modify this code to extract relevant information from your documents
        summary = doc.body[:200] + '...'  # Example summary: First 200 characters of the body
        summaries.append(summary)

    return render_template('results.html', query=query, results=results, summaries=summaries)

if __name__ == '__main__':
    app.run(debug=True)






Page saved: Cricket.html
Page saved: Cricket_(insect).html
Page saved: Cricket_(disambiguation).html
Page saved: New_York_City.html
Page saved: Box_score_(baseball).html
Page saved: Cricket#cite_note-158.html
Page saved: New_York_Clipper.html
Page saved: Cricket#cite_note-NPR-159.html
Page saved: Cricket#cite_note-Myth-160.html
Page saved: Portal:Cricket.html
Page saved: Cricket#bodyContent.html
Page saved: Main_Page.html
Page saved: Wikipedia:Contents.html
Page saved: Portal:Current_events.html
Page saved: Special:Random.html
Page saved: Wikipedia:About.html
Page saved: Wikipedia:Contact_us.html
Page saved: Special:FundraiserRedirector?utm_source=donate&utm_medium=sidebar&utm_campaign=C13_en.wikipedia.org&uselang=en.html
Page saved: Help:Contents.html
Page saved: Help:Introduction.html
Page saved: Wikipedia:Community_portal.html
Page saved: Special:RecentChanges.html
Page saved: Wikipedia:File_upload_wizard.html
Page saved: Special:Search.html
Page saved: index.php?title=Special:Creat

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
from flask import Flask, request, render_template
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
from bs4 import BeautifulSoup
import requests
import os

app = Flask(__name__)

class WebCrawler:
    def __init__(self, seeds):
        self.seeds = seeds
        self.url_frontier = list(seeds)
        self.visited_urls = set()
        self.crawled_pages = []

    def crawl(self, max_pages):
        while self.url_frontier and len(self.crawled_pages) < max_pages:
            url = self.url_frontier.pop(0)
            if url not in self.visited_urls:
                content = self.fetch_page_content(url)
                if content is not None:
                    self.save_page(url, content)
                    new_links = self.extract_links(url, content)
                    self.add_new_links(new_links)
                    self.visited_urls.add(url)

    def fetch_page_content(self, url):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                return response.text
        except requests.exceptions.RequestException as e:
            print("Error fetching page:", e)
        return None

    def save_page(self, url, content):
        self.crawled_pages.append((url, content))
        filename = os.path.basename(url) + ".html"  # Extract filename from the URL
        with open(filename, "w", encoding="utf-8") as file:
            file.write(content)
        print("Page saved:", filename)

    def extract_links(self, url, content):
        links = []
        soup = BeautifulSoup(content, 'html.parser')
        for link in soup.find_all('a'):
            href = link.get('href')
            if href:
                absolute_url = urljoin(url, href)
                links.append(absolute_url)
        return links

    def add_new_links(self, links):
        for link in links:
            if link not in self.visited_urls and link not in self.url_frontier:
                self.url_frontier.append(link)

# Example usage
seeds = ['https://en.wikipedia.org/wiki/Cricket']
max_pages = 100
crawler = WebCrawler(seeds)
crawler.crawl(max_pages)

# Text Extraction and Preprocessing
documents = []
for url, content in crawler.crawled_pages:
    soup = BeautifulSoup(content, 'html.parser')
    title = soup.find('title')
    body = soup.find('body')
    if title and body:
        document = title.get_text() + " " + body.get_text()
        documents.append(document)

# Indexing
class Indexer:
    def __init__(self):
        self.vectorizer = None
        self.index = None
        self.document_urls = []

    def build_index(self, documents):
        self.vectorizer = TfidfVectorizer()
        self.index = self.vectorizer.fit_transform(documents)

    def get_document_urls(self):
        return self.document_urls

indexer = Indexer()
if documents:
    indexer.build_index(documents)
    indexer.document_urls = [url for url, _ in crawler.crawled_pages]
else:
    print("No documents found for indexing.")

# Search Engine
class SearchEngine:
    def __init__(self, indexer):
        self.indexer = indexer

    def search(self, query, ranking_method):
        query_vector = self.indexer.vectorizer.transform([query])

        if ranking_method == 'cosine':
            scores = cosine_similarity(query_vector, self.indexer.index).flatten()
        elif ranking_method == 'bm25':
            bm25 = BM25Okapi(self.indexer.index)
            scores = bm25.get_scores(query_vector)

        top_indices = scores.argsort()[::-1][:10]
        top_urls = [self.indexer.document_urls[i] for i in top_indices]
        return top_urls

search_engine = SearchEngine(indexer)

@app.route('/', methods=['GET', 'POST'])
def search():
    if request.method == 'POST':
        query = request.form['query']
        ranking_method = request.form['ranking_method']
        results = search_engine.search(query, ranking_method)
        print("Results:", results)  # Print the results
        return render_template('results.html', query=query, results=results)
    return render_template('index.html')

'''def search():
    if request.method == 'POST':
        query = request.form['query']
        ranking_method = request.form['ranking_method']
        results = search_engine.search(query, ranking_method)
        return render_template('results.html', query=query, results=results)
    return render_template('index.html')'''

if __name__ == '__main__':
    app.run(debug=True)


Page saved: Cricket.html
Page saved: Cricket#bodyContent.html
Page saved: Main_Page.html
Page saved: Wikipedia:Contents.html
Page saved: Portal:Current_events.html
Page saved: Special:Random.html
Page saved: Wikipedia:About.html
Page saved: Wikipedia:Contact_us.html
Page saved: Special:FundraiserRedirector?utm_source=donate&utm_medium=sidebar&utm_campaign=C13_en.wikipedia.org&uselang=en.html
Page saved: Help:Contents.html
Page saved: Help:Introduction.html
Page saved: Wikipedia:Community_portal.html
Page saved: Special:RecentChanges.html
Page saved: Wikipedia:File_upload_wizard.html
Page saved: Special:Search.html
Page saved: index.php?title=Special:CreateAccount&returnto=Cricket.html
Page saved: index.php?title=Special:UserLogin&returnto=Cricket.html
Page saved: Special:MyContributions.html
Page saved: Special:MyTalk.html
Page saved: Cricket#History.html
Page saved: Cricket#Origins.html
Page saved: Cricket#Growth_of_amateur_and_professional_cricket_in_England.html
Page saved: Cricket#

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
